Load dataset:

In [1]:
from HumanGenomeDataset.load_dataset import load_dataset

data_df = load_dataset('dna_protein_coding_sequences')
data_df.head(2)

,ID,sequence,category
0,NM_001368254.1,GCTGAGCTGAGCTGGGGCGCAGCCGCCTGTCTGCACCGGCAGCACC...,NM
1,NM_001350977.1,TCCACAACTGAAACATCCACTTCTGAACACCATGTCCTACTACAGC...,NM


Start simulation:

In [2]:
from src.simulation import ProteinSinthesisProcess
SIM_TIME = 1000

In [3]:
protein_synthesis_process = ProteinSinthesisProcess(dna_sequences_df=data_df, verbose=True)
protein_synthesis_process.run(simulation_time=SIM_TIME) # run the simulation

Eucaryotes cell initialized
Simulation environment initialized 	
Simulation started: 	
Protein synthesized: 1
Protein synthesized: 0
Protein synthesized: 11
Protein synthesized: 14
Protein synthesized: 6
Protein synthesized: 3
Protein synthesized: 1
Protein synthesized: 0
Protein synthesized: 1
Protein synthesized: 0
Protein synthesized: 0
Protein synthesized: 2
Protein synthesized: 0
Protein synthesized: 14
Protein synthesized: 4
Protein synthesized: 0
Protein synthesized: 3
Protein synthesized: 0
Protein synthesized: 7
Protein synthesized: 1
Protein synthesized: 7
Protein synthesized: 0
Protein synthesized: 0
Protein synthesized: 1
Protein synthesized: 5
Protein synthesized: 2
Protein synthesized: 1
Protein synthesized: 3
Protein synthesized: 3
Protein synthesized: 17
Protein synthesized: 0
Protein synthesized: 0
Protein synthesized: 1
Protein synthesized: 0
Protein synthesized: 10
Protein synthesized: 4
Protein synthesized: 1
Protein synthesized: 0
Protein synthesized: 9
Protein syn

IndexError: index 0 is out of bounds for axis 0 with size 0

Results:

In [4]:
results_df = protein_synthesis_process.dna_sequences_df

In [5]:
results_df[results_df['mrna_sequences'].notna()][
    ['polypeptides_chains', 'polypeptides_chains_ext', 'number_of_proteins_synthesized']]

,polypeptides_chains,polypeptides_chains_ext,number_of_proteins_synthesized
582,"[NH2-ERVVASTV-COOH, NH2-NCYN-COOH]","[NH2-Glu-Arg-Val-Val-Ala-Ser-Thr-Val-COOH, NH2...",2
835,"[NH2--COOH, NH2-NLIEYNREDSK-COOH]","[NH2-COOH, NH2-Asn-Leu-Ile-Glu-Tyr-Asn-Arg-Glu...",2
1065,[NH2-FSKTRIFFCFFCFLFFRSDRVPSTRKVMWDYNFGTITLNTG...,[NH2-Phe-Ser-Lys-Thr-Arg-Ile-Phe-Phe-Cys-Phe-P...,17
2174,"[NH2-YIV-COOH, NH2-CTCLQ-COOH, NH2-SLRPEFGKKRY...","[NH2-Tyr-Ile-Val-COOH, NH2-Cys-Thr-Cys-Leu-Gln...",7
2605,"[NH2-DDVLDLQSTLTT-COOH, NH2-SDLVRGGVYLGSQHHL-C...",[NH2-Asp-Asp-Val-Leu-Asp-Leu-Gln-Ser-Thr-Leu-T...,3
...,...,...,...
178008,"[NH2-FFSFSFRP-COOH, NH2-SHPNNYFNSPTLWEYLVTVVPY...","[NH2-Phe-Phe-Ser-Phe-Ser-Phe-Arg-Pr-COOH, NH2-...",2
179729,"[NH2--COOH, NH2-NLHSWKMRSYINYMSKNNYLL-COOH, NH...","[NH2-COOH, NH2-Asn-Leu-His-Ser-Trp-Lys-Met-Arg...",4
182482,[NH2-SILILNGQTEGCTGVRTLEGSWYRQDETELEPPTCLW-COOH],[NH2-Ser-Ile-Leu-Ile-Leu-Asn-Gly-Gln-Thr-Glu-G...,1
183767,"[NH2-TFITL-COOH, NH2--COOH, NH2-IFMS-COOH, NH2...","[NH2-Thr-Phe-Ile-Thr-Leu-COOH, NH2-COOH, NH2-I...",6
